# Week 4: Limits, Continuity, and Introduction to Derivatives

## Numerical Differentiation Using Cubic Splines

**SCIE1500 - Analytical Skills for Science Students**

---

### Learning Objectives

In this notebook, you will learn to:

1. Understand **limits** conceptually and compute them numerically
2. Visualize **continuity** and types of discontinuity
3. Understand **derivatives** as instantaneous rates of change
4. Use **cubic spline interpolation** to compute derivatives numerically
5. Draw **tangent lines** at specified points on a curve
6. Connect differentiation to the **Schaefer model** from Week 3

### Exam Alignment

This notebook prepares you for exam questions:
- **Q21**: Tangent line to exponential function
- **Q25**: Properties of polynomial functions (slope via derivative)
- **Q28**: Differentiation using power rule and logarithm
- **Q13**: Schaefer model - setting G'(S) = 0 to find MSY stock level

---

In [ ]:
# Standard imports for Week 4
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.interpolate import CubicSpline

# Set plot style for cleaner visuals
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = [10, 6]
plt.rcParams['font.size'] = 11

print("Week 4 imports ready: NumPy, Matplotlib, Pandas, SciPy CubicSpline")

---

## Part A: Understanding Limits Numerically

### What is a Limit?

The **limit** of $f(x)$ as $x$ approaches $a$ is the value that $f(x)$ gets arbitrarily close to as $x$ gets arbitrarily close to $a$:

$$\lim_{x \to a} f(x) = L$$

### Why Limits Matter

Consider the expression $\frac{x^2 - 4}{x - 2}$. At $x = 2$, we get $\frac{0}{0}$ — **indeterminate**!

But what happens as $x$ *approaches* 2?

In [ ]:
def compute_limit_table(f, a, label='f(x)'):
    """Compute function values approaching a point from both sides.
    
    Args:
        f: function to evaluate
        a: point to approach
        label: name for the function in output
    """
    # Values approaching from the left (a⁻) and right (a⁺)
    h_values = np.array([0.1, 0.01, 0.001, 0.0001, 0.00001])
    x_left = a - h_values
    x_right = a + h_values
    y_left = f(x_left)
    y_right = f(x_right)
    
    print(f"\nLimit of {label} as x → {a}")
    print("=" * 55)
    print(f"{'From Left (x → a⁻)':^27} | {'From Right (x → a⁺)':^27}")
    print(f"{'x':^12} {'f(x)':^14} | {'x':^12} {'f(x)':^14}")
    print("-" * 55)
    
    for i in range(len(h_values)):
        print(f"{x_left[i]:12.6f} {y_left[i]:14.6f} | {x_right[i]:12.6f} {y_right[i]:14.6f}")
    
    print("-" * 55)
    print(f"Left limit ≈ {y_left[-1]:.6f}")
    print(f"Right limit ≈ {y_right[-1]:.6f}")
    
    if np.isclose(y_left[-1], y_right[-1], rtol=1e-4):
        print(f"\n✓ Limit exists: lim(x→{a}) {label} ≈ {y_left[-1]:.6f}")
    else:
        print(f"\n✗ Limit does not exist (left ≠ right)")

# Example: lim(x→2) (x² - 4)/(x - 2)
def f_indeterminate(x):
    return (x**2 - 4) / (x - 2)

compute_limit_table(f_indeterminate, 2, '(x² - 4)/(x - 2)')

**Key Insight:** Even though $f(2)$ is undefined (0/0), the limit exists and equals **4**.

This is because $\frac{x^2 - 4}{x - 2} = \frac{(x+2)(x-2)}{x-2} = x + 2$ for $x \neq 2$.

So $\lim_{x \to 2} (x+2) = 4$.

In [ ]:
# Visualize the removable discontinuity
x = np.linspace(-1, 5, 200)
# Avoid x = 2
x = x[np.abs(x - 2) > 0.01]
y = (x**2 - 4) / (x - 2)

plt.figure(figsize=(10, 6))
plt.plot(x, y, 'b-', linewidth=2, label='$f(x) = \\frac{x^2-4}{x-2}$')
plt.plot(2, 4, 'ro', markersize=10, markerfacecolor='white', markeredgewidth=2, label='Hole at (2, 4)')
plt.xlabel('x', fontsize=12)
plt.ylabel('y', fontsize=12)
plt.title('Removable Discontinuity: Limit Exists But Function Undefined', fontsize=14)
plt.legend()
plt.grid(True, alpha=0.3)
plt.xlim(-1, 5)
plt.ylim(0, 8)
plt.show()

print("Notice: The function is just y = x + 2 with a 'hole' at x = 2")

### Practice: Limits by Factoring (Exam Style)

**Example (W4-001 style):** Evaluate $\lim_{x \to 3} \frac{x^2 - 9}{x - 3}$

In [ ]:
# Verify numerically
def f_example(x):
    return (x**2 - 9) / (x - 3)

compute_limit_table(f_example, 3, '(x² - 9)/(x - 3)')

print("\n--- Algebraic Solution ---")
print("Factor: x² - 9 = (x+3)(x-3)")
print("Cancel: (x+3)(x-3)/(x-3) = x + 3  (for x ≠ 3)")
print("Substitute: lim(x→3)(x+3) = 3 + 3 = 6 ✓")

---

## Part B: The Derivative — From Average to Instantaneous Rate

### The Core Idea

The **derivative** is defined as:

$$f'(x) = \lim_{h \to 0} \frac{f(x+h) - f(x)}{h}$$

This is the slope of the **tangent line** — the instantaneous rate of change.

Let's watch the secant line approach the tangent as $h \to 0$:

In [ ]:
def f(x):
    """Example function: f(x) = x²"""
    return x**2

def secant_slope(f, a, h):
    """Calculate slope of secant line through (a, f(a)) and (a+h, f(a+h))"""
    return (f(a + h) - f(a)) / h

# Point of tangency
a = 2
true_slope = 2 * a  # f'(x) = 2x, so f'(2) = 4

# Various values of h (getting smaller)
h_values = [2.0, 1.0, 0.5, 0.25, 0.1, 0.01]

# Create plot
fig, ax = plt.subplots(figsize=(12, 8))

# Plot the function
x = np.linspace(0, 5, 200)
ax.plot(x, f(x), 'b-', linewidth=2, label='$f(x) = x^2$')

# Plot secant lines for each h
colors = plt.cm.Reds(np.linspace(0.3, 1, len(h_values)))
for i, h in enumerate(h_values):
    slope = secant_slope(f, a, h)
    # Secant line: y - f(a) = slope * (x - a)
    x_line = np.linspace(a - 1, a + h + 0.5, 50)
    y_line = f(a) + slope * (x_line - a)
    ax.plot(x_line, y_line, '--', color=colors[i], alpha=0.7, 
            label=f'h = {h}: slope = {slope:.3f}')
    # Mark the two points
    ax.plot([a, a+h], [f(a), f(a+h)], 'o', color=colors[i], markersize=6)

# True tangent line
x_tangent = np.linspace(a - 1, a + 2.5, 50)
y_tangent = f(a) + true_slope * (x_tangent - a)
ax.plot(x_tangent, y_tangent, 'g-', linewidth=3, label=f'Tangent: slope = {true_slope:.0f}')
ax.plot(a, f(a), 'go', markersize=12, zorder=5)

ax.set_xlabel('x', fontsize=12)
ax.set_ylabel('y', fontsize=12)
ax.set_title('Secant Lines Approaching Tangent Line as h → 0', fontsize=14)
ax.legend(loc='upper left', fontsize=10)
ax.set_xlim(0, 5)
ax.set_ylim(0, 20)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nAs h → 0, the secant slope → derivative f'({a}) = {true_slope}")

### Computing the Derivative Numerically

Let's see how the difference quotient converges to the derivative:

In [ ]:
# Derivative of f(x) = x² at x = 3 using limit definition
a = 3

def derivative_limit(h):
    """Compute [f(a+h) - f(a)] / h for f(x) = x²"""
    return ((a + h)**2 - a**2) / h

print("Derivative of f(x) = x² at x = 3 using limit definition")
print("=" * 50)
print(f"\n{'h':^15} {'[f(3+h) - f(3)] / h':^20}")
print("-" * 35)

h_values = [1, 0.1, 0.01, 0.001, 0.0001, 0.00001]
for h in h_values:
    print(f"{h:15.6f} {derivative_limit(h):20.6f}")

print(f"\nAs h → 0, the limit → 6 = 2×3 = f'(3) ✓")
print("\nThis confirms the power rule: d/dx[x²] = 2x")

---

## Part C: Cubic Spline Differentiation

### Why Use Cubic Splines?

As scientists, we often have **data** without knowing the underlying function. How do we find derivatives?

**Cubic spline interpolation** fits smooth curves through data points and can compute derivatives at any point.

Let's use an example with **lifespan vs metabolic rate** data:

In [ ]:
# Lifespan data (from mammals)
lifespan_data = {
    'animal': ['horse', 'sheep', 'red fox', 'raccoon', 'hedgehog', 'rat', 'mouse', 'mole'],
    'mrate': [0.18, 0.28, 0.52, 0.68, 0.75, 0.86, 1.36, 1.64],  # metabolic rate (cm³ O₂/g/h)
    'lifespan': [46, 20, 9.8, 14, 6, 4.7, 3.5, 4.5]  # years
}
lifespan_df = pd.DataFrame(lifespan_data)
print("Lifespan Data:")
print(lifespan_df.to_string(index=False))

In [ ]:
# Create cubic spline
lifespan_cs = CubicSpline(lifespan_df['mrate'], lifespan_df['lifespan'])

# Plot data and spline fit
mrate_fine = np.linspace(0.15, 1.9, 200)
lifespan_pred = lifespan_cs(mrate_fine)

plt.figure(figsize=(10, 7))
plt.plot(mrate_fine, lifespan_pred, 'b-', linewidth=2, label='Cubic Spline Fit')
plt.plot(lifespan_df['mrate'], lifespan_df['lifespan'], 'ro', markersize=8, label='Data Points')

# Add labels for animals
for i, row in lifespan_df.iterrows():
    plt.annotate(row['animal'], (row['mrate'], row['lifespan']), 
                 textcoords='offset points', xytext=(5, 5), fontsize=9)

plt.xlabel('Metabolic Rate ($cm^3$ $O_2$/g/h)', fontsize=12)
plt.ylabel('Lifespan (years)', fontsize=12)
plt.title('Mammal Lifespan vs Metabolic Rate', fontsize=14)
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print("\nKey feature: The cubic spline passes through ALL data points exactly!")

### Using the Spline to Compute Derivatives

The cubic spline has a `.derivative()` method that returns the **derivative function**:

In [ ]:
# Get the derivative function from the cubic spline
lifespan_cs_deriv = lifespan_cs.derivative(1)  # 1 = first derivative

# Calculate slope at various metabolic rates
test_mrates = [0.30, 0.52, 0.75, 1.00, 1.50]

print("How does lifespan change with metabolic rate?")
print("=" * 55)
print(f"{'Metabolic Rate':^15} {'Predicted Lifespan':^20} {'Slope (dy/dx)':^15}")
print("-" * 55)

for mrate in test_mrates:
    lifespan = lifespan_cs(mrate)
    slope = lifespan_cs_deriv(mrate)
    print(f"{mrate:^15.2f} {lifespan:^20.2f} {slope:^15.2f}")

print("\nInterpretation: Negative slopes mean higher metabolic rate → shorter lifespan")

---

## Part D: Drawing Tangent Lines

### Tangent Line Formula

At point $(a, f(a))$, the tangent line has:
- **Slope:** $m = f'(a)$
- **Equation:** $y = f(a) + f'(a)(x - a)$

Let's work with a known function: $y = 3x^2 - 4x + 5$

In [ ]:
def f(x):
    """Function: f(x) = 3x² - 4x + 5"""
    return 3*x**2 - 4*x + 5

def f_prime(x):
    """Analytical derivative: f'(x) = 6x - 4"""
    return 6*x - 4

# Create x and y values for the cubic spline
xvalues = np.linspace(-10, 10, 100)
yvalues = f(xvalues)

# Create cubic spline and its derivative
f_cs = CubicSpline(xvalues, yvalues)
f_cs_deriv = f_cs.derivative(1)

# Point of tangency
x0 = 5
y0 = f_cs(x0)
slope = f_cs_deriv(x0)

print("Tangent Line Calculation for y = 3x² - 4x + 5 at x = 5")
print("=" * 55)
print(f"\nStep 1: Find the point")
print(f"  f({x0}) = 3({x0})² - 4({x0}) + 5 = {3*x0**2} - {4*x0} + 5 = {y0:.1f}")
print(f"  Point: ({x0}, {y0:.1f})")
print(f"\nStep 2: Find the slope (derivative)")
print(f"  f'(x) = 6x - 4")
print(f"  f'({x0}) = 6({x0}) - 4 = {slope:.1f}")
print(f"\nStep 3: Tangent line equation")
print(f"  y - {y0:.1f} = {slope:.1f}(x - {x0})")
print(f"  y = {slope:.1f}x + {y0 - slope*x0:.1f}")

In [ ]:
# Plot function and tangent line
x_plot = np.linspace(-2, 8, 200)
y_plot = f(x_plot)

# Tangent line
x_tangent = np.linspace(x0 - 2, x0 + 2, 50)
y_tangent = y0 + slope * (x_tangent - x0)

plt.figure(figsize=(10, 7))
plt.plot(x_plot, y_plot, 'b-', linewidth=2, label='$f(x) = 3x^2 - 4x + 5$')
plt.plot(x_tangent, y_tangent, 'r--', linewidth=2, label=f'Tangent: y = {slope:.0f}x + {y0 - slope*x0:.0f}')
plt.plot(x0, y0, 'ro', markersize=12, zorder=5)
plt.annotate(f'({x0}, {y0:.0f})\nslope = {slope:.0f}', 
             xy=(x0, y0), xytext=(x0-2, y0+15),
             fontsize=11, arrowprops=dict(arrowstyle='->', color='red'))

plt.xlabel('x', fontsize=12)
plt.ylabel('y', fontsize=12)
plt.title('Function and Tangent Line at x = 5', fontsize=14)
plt.legend(loc='upper left', fontsize=11)
plt.grid(True, alpha=0.3)
plt.xlim(-2, 8)
plt.ylim(0, 100)
plt.tight_layout()
plt.show()

---

## Part E: The Power Rule and Verification

### The Power Rule

$$\frac{d}{dx}[x^n] = nx^{n-1}$$

This works for **any real** $n$ — positive, negative, or fractional.

Let's verify this numerically:

In [ ]:
def power_rule_check(n, x0, h=0.0001):
    """Verify power rule d/dx[x^n] = n*x^(n-1) at point x0"""
    
    def f(x):
        return x**n
    
    # Numerical derivative (central difference for better accuracy)
    numerical = (f(x0 + h) - f(x0 - h)) / (2 * h)
    
    # Exact derivative using power rule
    exact = n * x0**(n - 1)
    
    return numerical, exact

print("Power Rule Verification: d/dx[x^n] = n·x^(n-1)")
print("=" * 65)

test_cases = [
    (2, 3, "x² at x=3: should give 2×3¹ = 6"),
    (3, 2, "x³ at x=2: should give 3×2² = 12"),
    (0.5, 4, "√x at x=4: should give 0.5×4^(-0.5) = 0.25"),
    (-1, 2, "1/x at x=2: should give -1×2^(-2) = -0.25"),
    (5, 1, "x⁵ at x=1: should give 5×1⁴ = 5"),
]

print(f"\n{'n':^6} {'x₀':^6} {'Numerical':^12} {'Exact':^12} {'Description'}")
print("-" * 65)

for n, x0, desc in test_cases:
    num, exact = power_rule_check(n, x0)
    print(f"{n:^6.1f} {x0:^6.1f} {num:^12.6f} {exact:^12.6f} {desc}")

print("\n✓ Numerical approximations match exact values (power rule verified!)")

---

## Part F: Schaefer Model — Connecting to Week 3

### Differentiating the Schaefer Growth Model

Recall from Week 3 the Schaefer growth model:

$$G(S) = gS\left(1 - \frac{S}{K}\right) = gS - \frac{g}{K}S^2$$

**Question (Exam Q13 style):** At what stock level $S^*$ is growth maximized?

**Answer:** We need to find where $G'(S) = 0$.

In [ ]:
# Schaefer model parameters
g = 0.12  # intrinsic growth rate
K = 10000  # carrying capacity

def schaefer_growth(S, g, K):
    """Schaefer growth function G(S)"""
    return g * S * (1 - S/K)

def schaefer_derivative(S, g, K):
    """Analytical derivative: G'(S) = g - 2gS/K"""
    return g - 2*g*S/K

# Create stock values and compute growth
S_values = np.linspace(0, K, 200)
G_values = schaefer_growth(S_values, g, K)
G_deriv_values = schaefer_derivative(S_values, g, K)

# Find MSY using derivative = 0
# G'(S*) = g - 2gS*/K = 0  =>  S* = K/2
S_MSY = K / 2
G_MSY = schaefer_growth(S_MSY, g, K)

print("Schaefer Model Analysis")
print("=" * 50)
print(f"Parameters: g = {g}, K = {K}")
print(f"\nGrowth function: G(S) = gS(1 - S/K) = {g}S(1 - S/{K})")
print(f"\nExpanded: G(S) = {g}S - {g/K}S²")
print(f"\nDerivative: G'(S) = {g} - {2*g/K}S")
print(f"\nSetting G'(S*) = 0:")
print(f"  {g} - {2*g/K}S* = 0")
print(f"  S* = {g} / {2*g/K} = {S_MSY:.0f}")
print(f"\n✓ MSY Stock Level: S* = K/2 = {S_MSY:.0f} tonnes")
print(f"✓ Maximum Growth: G(S*) = gK/4 = {G_MSY:.0f} tonnes/year")

In [ ]:
# Visualize the Schaefer model and its derivative
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Left plot: Growth function
ax1.plot(S_values, G_values, 'b-', linewidth=2, label='$G(S) = gS(1-S/K)$')
ax1.axvline(x=S_MSY, color='r', linestyle='--', label=f'$S_{{MSY}} = K/2 = {S_MSY:.0f}$')
ax1.plot(S_MSY, G_MSY, 'ro', markersize=12, zorder=5)
ax1.annotate(f'MSY = {G_MSY:.0f}', xy=(S_MSY, G_MSY), xytext=(S_MSY+1000, G_MSY+20),
             fontsize=11, arrowprops=dict(arrowstyle='->', color='red'))
ax1.set_xlabel('Stock S (tonnes)', fontsize=12)
ax1.set_ylabel('Growth G(S) (tonnes/year)', fontsize=12)
ax1.set_title('Schaefer Growth Model', fontsize=14)
ax1.legend()
ax1.grid(True, alpha=0.3)

# Right plot: Derivative
ax2.plot(S_values, G_deriv_values, 'g-', linewidth=2, label="$G'(S) = g - 2gS/K$")
ax2.axhline(y=0, color='k', linestyle='-', alpha=0.3)
ax2.axvline(x=S_MSY, color='r', linestyle='--', label=f'$G\'(S_{{MSY}}) = 0$')
ax2.plot(S_MSY, 0, 'ro', markersize=12, zorder=5)
ax2.fill_between(S_values, G_deriv_values, 0, where=(G_deriv_values > 0), 
                  alpha=0.3, color='green', label='Growth increasing')
ax2.fill_between(S_values, G_deriv_values, 0, where=(G_deriv_values < 0), 
                  alpha=0.3, color='red', label='Growth decreasing')
ax2.set_xlabel('Stock S (tonnes)', fontsize=12)
ax2.set_ylabel("G'(S)", fontsize=12)
ax2.set_title('Derivative of Growth Function', fontsize=14)
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Key Insight: Where G'(S) = 0, growth is maximized (MSY occurs at S = K/2)")

---

## Part G: Exam-Aligned Practice

### Exam Q21 Style: Tangent to Exponential Function

**Problem:** Find the equation of the tangent to $y = e^{x-2} + 3$ at $x = 2$.

In [ ]:
# Q21: Tangent to y = e^(x-2) + 3 at x = 2
print("Exam Q21 Solution: Tangent to y = e^(x-2) + 3 at x = 2")
print("=" * 55)

# Step 1: Find the point
x0 = 2
y0 = np.exp(x0 - 2) + 3  # e^0 + 3 = 1 + 3 = 4
print(f"\nStep 1: Find the point")
print(f"  y(2) = e^(2-2) + 3 = e^0 + 3 = 1 + 3 = {y0}")
print(f"  Point: ({x0}, {y0})")

# Step 2: Find the derivative
# y = e^(x-2) + 3, so y' = e^(x-2) (chain rule, derivative of constant = 0)
slope = np.exp(x0 - 2)  # e^0 = 1
print(f"\nStep 2: Find the derivative")
print(f"  y' = e^(x-2) (the +3 constant vanishes)")
print(f"  y'(2) = e^(2-2) = e^0 = {slope}")

# Step 3: Write tangent equation
print(f"\nStep 3: Write the tangent line equation")
print(f"  y - {y0} = {slope}(x - {x0})")
print(f"  y = x - 2 + 4")
print(f"  y = x + 2")
print(f"\n✓ Answer: y = x + 2 (Option D)")

In [ ]:
# Visualize Q21 solution
x_plot = np.linspace(-1, 5, 200)
y_func = np.exp(x_plot - 2) + 3
y_tangent = x_plot + 2

plt.figure(figsize=(10, 7))
plt.plot(x_plot, y_func, 'b-', linewidth=2, label='$y = e^{x-2} + 3$')
plt.plot(x_plot, y_tangent, 'r--', linewidth=2, label='Tangent: $y = x + 2$')
plt.plot(2, 4, 'ro', markersize=12, zorder=5)
plt.annotate('(2, 4)\nslope = 1', xy=(2, 4), xytext=(3, 6),
             fontsize=11, arrowprops=dict(arrowstyle='->', color='red'))

plt.xlabel('x', fontsize=12)
plt.ylabel('y', fontsize=12)
plt.title('Exam Q21: Tangent to $y = e^{x-2} + 3$ at $x = 2$', fontsize=14)
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.xlim(-1, 5)
plt.ylim(0, 15)
plt.tight_layout()
plt.show()

### Exam Q25 Style: Properties of Polynomial

**Problem:** For $y = (x+2)(x-3)$, find the slope at $x = 2$.

In [ ]:
# Q25: Properties of y = (x+2)(x-3)
print("Exam Q25 Style: y = (x+2)(x-3) = x² - x - 6")
print("=" * 50)

print("\nExpand: y = x² - x - 6")
print("\nDerivative: y' = 2x - 1")
print("\nSlope at x = 2: y'(2) = 2(2) - 1 = 3")

# Verify numerically
x_vals = np.linspace(-3, 5, 100)
y_vals = (x_vals + 2) * (x_vals - 3)

cs = CubicSpline(x_vals, y_vals)
cs_deriv = cs.derivative(1)

numerical_slope = cs_deriv(2)
print(f"\nNumerical verification: slope at x=2 is {numerical_slope:.4f}")
print("\n✓ Statement C in Q25 is TRUE: slope at x=2 is 3")

---

## Student Exercises

### Exercise A (Show Demonstrator): Tangent Line Practice

For the function $f(x) = 2x^3 - 5x^2 + 3x - 1$:

1. Create a cubic spline approximation
2. Find the slope (derivative) at $x = 2$
3. Find the tangent line equation at $x = 2$
4. Plot the function and tangent line

**Expected:** f'(2) = 11, tangent: y = 11x - 17

In [ ]:
# STUDENT EXERCISE A: Complete the code below

# Step 1: Define the function
def f(x):
    return 2*x**3 - 5*x**2 + 3*x - 1

# Step 2: Create x values and compute y values
xvalues = np.linspace(-2, 5, 100)
yvalues = # YOUR CODE HERE

# Step 3: Create cubic spline and derivative
f_cs = # YOUR CODE HERE
f_cs_deriv = # YOUR CODE HERE

# Step 4: Find point and slope at x = 2
x0 = 2
y0 = # YOUR CODE HERE
slope = # YOUR CODE HERE

print(f"Point of tangency: ({x0}, {y0:.1f})")
print(f"Slope at x = {x0}: {slope:.1f}")
print(f"Tangent line: y = {slope:.1f}x + {y0 - slope*x0:.1f}")

# Step 5: Plot (uncomment and complete)
# plt.figure(figsize=(10, 7))
# plt.plot(xvalues, yvalues, 'b-', linewidth=2, label='f(x)')
# x_tangent = np.linspace(x0 - 2, x0 + 2, 50)
# y_tangent = # YOUR CODE HERE
# plt.plot(x_tangent, y_tangent, 'r--', linewidth=2, label='Tangent')
# plt.plot(x0, y0, 'ro', markersize=10)
# plt.xlabel('x')
# plt.ylabel('y')
# plt.title('Exercise A: Tangent Line')
# plt.legend()
# plt.grid(True, alpha=0.3)
# plt.show()

### Exercise B: Cubic Spline Differentiation

For the function $y = \sqrt{\frac{x^2 + 1}{x^4 + 1}}$:

1. Create 201 x values from -10 to 10
2. Create a cubic spline
3. Get the derivative function
4. Plot both $y$ and $y'$ on the same graph

Use GREEN for the function and RED for the derivative.

In [ ]:
# STUDENT EXERCISE B: Complete the code below

# Define function
def f(x):
    return np.sqrt((x**2 + 1) / (x**4 + 1))

# Create x values
xvalues = # YOUR CODE HERE (201 values from -10 to 10)

# Calculate y values
yvalues = # YOUR CODE HERE

# Create cubic spline
# YOUR CODE HERE

# Get derivative function
# YOUR CODE HERE

# Calculate derivative values
slopevalues = # YOUR CODE HERE

# Plot both curves
# plt.figure(figsize=(12, 6))
# plt.plot(xvalues, yvalues, 'g-', linewidth=2, label='f(x)')
# plt.plot(xvalues, slopevalues, 'r-', linewidth=2, label="f'(x)")
# plt.xlabel('x', fontsize=12)
# plt.ylabel('y', fontsize=12)
# plt.title('Function and Derivative', fontsize=14)
# plt.legend()
# plt.grid(True, alpha=0.3)
# plt.show()

### Exercise C (Upload): Schaefer Model Analysis

A fishery has the following parameters:
- Intrinsic growth rate: $g = 0.15$
- Carrying capacity: $K = 8000$ tonnes

Using the Schaefer model $G(S) = gS(1 - S/K)$:

1. Calculate the derivative $G'(S)$
2. Find the MSY stock level by setting $G'(S) = 0$
3. Calculate the maximum sustainable yield $G(S_{MSY})$
4. Plot both $G(S)$ and $G'(S)$ on the same graph
5. Verify the result: $S_{MSY} = K/2$ and $G_{MSY} = gK/4$

In [ ]:
# STUDENT EXERCISE C: Complete the Schaefer model analysis

# Parameters
g = 0.15
K = 8000

# Calculate MSY analytically
S_MSY = # YOUR CODE HERE
G_MSY = # YOUR CODE HERE

print(f"Schaefer Model Analysis")
print(f"=" * 40)
print(f"Parameters: g = {g}, K = {K}")
print(f"\nMSY Stock Level: S* = {S_MSY} tonnes")
print(f"Maximum Growth: G(S*) = {G_MSY} tonnes/year")

# Verify formulas
print(f"\nVerification:")
print(f"  K/2 = {K/2}")
print(f"  gK/4 = {g*K/4}")

# Create plot
# YOUR CODE HERE

---

## Summary: Key Concepts for Week 4

| Concept | Formula | Python Method |
|---------|---------|---------------|
| **Limit definition** | $\lim_{x \to a} f(x) = L$ | Numerical table |
| **Derivative definition** | $f'(x) = \lim_{h \to 0} \frac{f(x+h) - f(x)}{h}$ | `cs.derivative(1)` |
| **Power rule** | $\frac{d}{dx}[x^n] = nx^{n-1}$ | Analytical |
| **Tangent line** | $y = f(a) + f'(a)(x - a)$ | Point-slope form |
| **Schaefer MSY** | $G'(S^*) = 0 \Rightarrow S^* = K/2$ | Set derivative to 0 |

### What's Next: Week 5

Next week we'll use **SymPy** for symbolic differentiation and cover:
- Product rule, quotient rule, chain rule
- Derivatives of exponential and logarithmic functions
- **Optimization** — finding maxima and minima
- **Maximum Economic Yield (MEY)** in the bioeconomic fishery model